In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import *

In [2]:
def calc_shares(p1, p2,  cost, wtp):
    """compute the proportion of people choosing each insurer
    assumes interior solution"""
    s1 = np.clip(1/2 + (p2-p1)/(2*cost),0,1)
    s2 = 1-s1
    return s1,s2
  
def clip_p(p1, p2,  cost, wtp):
    s1,s2 = calc_shares(p1,p2,cost,wtp)
    p1_clip = np.clip(p1,0,wtp-s1*cost)
    p2_clip = np.clip(p2,0,wtp-s2*cost)
    return p1_clip,p2_clip
    


def calc_profits_price_shares(phi1,phi2,cost,wtp,mc):
    mc1,mc2 = mc
    
    clip_p1 = lambda p : clip_p(p[0],p[1],cost,wtp)
    
    pi = lambda p : -1*calc_shares(clip_p1(p)[0], clip_p1(p)[1],cost, wtp)[0]*(clip_p1(p)[0]-phi1 )\
                    + -1*calc_shares(clip_p1(p)[0], clip_p1(p)[1], cost, wtp)[1]*(clip_p1(p)[1]-phi2 )

    p = minimize(pi,[30,30]).x
    p1,p2 = p[0],p[1]  
    s1,s2 = calc_shares(p1, p2,  cost, wtp)
    return clip_p1(p)[0], clip_p1(p)[1], s1,s2, pi(p)

In [11]:
#arbitrary outside option...
def nash_in_nash_merge(phi1, phi2, cost, wtp, mc, beta=.5,active=False):
    p1,p2,s1,s2,pi_ins = calc_profits_price_shares(phi1,phi2,cost,wtp,mc)
    #print(p1,phi1,s1,s1*phi1,s1*(p1-phi1-mc[0]))
    obj = -1*(np.log(max(s1*phi1,1e-4))*(1-beta)  + np.log(s1*(p1-phi1-mc[0]))*beta)
    return obj


def bargain_helper_merge(phi1, phi2, cost, wtp, mc, beta=.5,active=False):
    """solve each firm 1s optimization holding phi 2 fixed"""
    result = minimize(nash_in_nash_merge, phi1, args=(phi2,cost, wtp, mc, beta, active),
                      method='Nelder-Mead', options={'disp': False})
    return result.x


def simult_bargain_merge(phi1, phi2, cost, wtp, mc, betas=[.5,.5],active=False):
    """solve the bargaining problems seperately and iterate for a solution"""       
    
    #loop variables, check on this...
    diff =  np.maximum(phi1,phi2)
    phi10,phi20 = 0,0
    maxiter = 10
    while maxiter >=0 and diff > 10e-7:
        #seems as though there is a contraction mapping here, need to think more about why
        phi1 = bargain_helper_merge(phi1, phi2, cost, wtp, mc, beta=betas[0] ,active=active)
        phi2 = bargain_helper_merge(phi2, phi1, cost, wtp, mc[::-1], beta=betas[1],active=active)
        
        #update loop variables
        diff = np.abs(np.maximum(phi1 - phi10,phi2-phi20))[0]
        phi10,phi20 = phi1,phi2
        maxiter = maxiter-1
        
    return phi1, phi2


COST = 5
WTP = 25
MC = np.array([0,0])


print(bargain_helper_merge(10, 10, COST, WTP, MC, beta=.5,active=False))


print(simult_bargain_merge(10, 10, COST, WTP, MC, betas=[.5,.5],active=False))

print(simult_bargain_merge(10, 10, COST, WTP, MC, betas=[.5,.1666],active=False))

[5.42700195]
(array([4.15179872]), array([4.15014235]))
(array([4.69052091]), array([6.45903317]))


In [27]:
WTP = 25
COST = 5
MC = np.array([0,0])
results = np.zeros(shape=(4,4))


BETA1 = [.5,.16666]
phis = simult_bargain(10, 10, COST, WTP, MC, betas=BETA1,active=True)
phi1,phi2 = phis[0][0],phis[1][0]
p1,p2 = calc_price(phi1, phi2, COST, WTP, MC)
print( np.array([[phi1,phi2,p1,p2]]).transpose() )
results[:,0] = [phi1,phi2,p1,p2]  #update results

phis = simult_bargain_merge(10, 10, COST, WTP, MC, betas=BETA1,active=True)
phi1,phi2 = phis[0][0],phis[1][0]
p1,p2,_,_,_ = calc_profits_price_shares(phi1,phi2,COST, WTP, MC)
results[:,1]  = [phi1,phi2,p1,p2]  #update results


BETA2 = [.5,.5]
phis = seq_bargain(20, COST, WTP, MC, betas=BETA2)
phi1,phi2 = phis[0][0],phis[1][0]
p1,p2 = calc_price(phi1, phi2, COST, WTP, MC)
results[:,2]  = [phi1,phi2,p1,p2]   #update results


phis = simult_bargain_merge(20, 20, COST, WTP, MC, betas=BETA2,active=True)
phi1,phi2 = phis[0][0],phis[1][0] 
p1,p2,_,_,_ = calc_profits_price_shares(phi1,phi2,COST, WTP, MC)
results[:,3]  = [phi1,phi2,p1,p2]   #update results

print(results)
for i in range(4):
    formatted_result = results[i,0],results[i,1],results[i,2],results[i,3]
    print("%.3f,%.3f,%.3f,%.3f"%(formatted_result))

[[15.62508373]
 [19.37517815]
 [21.8751152 ]
 [23.12514668]]


C:\Users\ericsc\AppData\Local\Temp\ipykernel_27688\659278290.py:5: RuntimeWarning: divide by zero encountered in log
  obj = -1*(np.log(max(s1*phi1,1e-4))*(1-beta)  + np.log(s1*(p1-phi1-mc[0]))*beta)


[[15.62508373  4.69052091 15.62661904  4.15094285]
 [19.37517815  6.45903317 19.37623908  4.15321596]
 [21.8751152  22.05787227 21.87649239 22.49943197]
 [23.12514668 22.94212773 23.12636573 22.50056803]]
15.625,4.691,15.627,4.151
19.375,6.459,19.376,4.153
21.875,22.058,21.876,22.499
23.125,22.942,23.126,22.501
